In [1]:
from cesnet_datazoo.datasets import CESNET_QUIC22
from cesnet_datazoo.config import DatasetConfig, AppSelection, ValidationApproach

dataset = CESNET_QUIC22("~/datasets/CESNET-QUIC22/", size="XS")

common_params = {
    "dataset": dataset,
    "apps_selection": AppSelection.ALL_KNOWN,
    "train_period_name": "W-2022-44",
    "val_approach": ValidationApproach.SPLIT_FROM_TRAIN,
    "train_val_split_fraction": 0.2,
    "use_packet_histograms": True,
}
dataset_config = DatasetConfig(**common_params)
dataset.set_dataset_config_and_initialize(dataset_config)
train_dataframe = dataset.get_train_df(flatten_ppi=True)
val_dataframe = dataset.get_val_df(flatten_ppi=True)
test_dataframe = dataset.get_test_df(flatten_ppi=True)

Loading data from dataloader


100%|██████████| 8162/8162 [00:08<00:00, 985.96it/s] 


Loading data from dataloader


100%|██████████| 192/192 [00:03<00:00, 48.07it/s]


Loading data from dataloader


100%|██████████| 1247/1247 [00:09<00:00, 133.86it/s]


In [52]:
from sklearn.neighbors import KNeighborsClassifier

import numpy as np

X_train = train_dataframe.drop(columns="APP").to_numpy()[:1000000]
y_train = train_dataframe["APP"].to_numpy()[:1000000]

X_test = test_dataframe.drop(columns="APP").to_numpy()[:100000]
y_test = test_dataframe["APP"].to_numpy()[:100000]

clf = KNeighborsClassifier(n_jobs=-1, n_neighbors=100)
clf.fit(X_train, y_train)

predictions = clf.predict_proba(X_test)

sums = [0, 0, 0, 0]

for i in range(len(predictions)):
    res_index = np.where(clf.classes_ == y_test[i])

    # print(predictions[i][res_index][0], end = " ")

    sum_other = -predictions[i][res_index][0]
    sum_correct = predictions[i][res_index][0]
    for index in range(len(predictions[i])):
        sum_other += predictions[i][index]

    coeff = sum_correct / sum_other

    if coeff < 0.1:
        sums[0] += 1
    elif coeff < 1:
        sums[1] += 1
    elif coeff < 10:
        sums[2] += 1
    else:
        sums[3] += 1

    # print(coeff)

print(sums)


/var/folders/8f/6dw3knc955311yk7rhd5t4980000gn/T/ipykernel_69878/263393245.py:28: RuntimeWarning: divide by zero encountered in scalar divide
  coeff = sum_correct / sum_other


[21364, 34055, 16992, 27589]
